## Detailed article explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541232/reducing-video-frames-and-frame-rates-fps-in-python

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing & Importing Required Libraries

In [1]:
! pip install deepface
! pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=ea25598a385f00c67e4f05127b62c04c97fa4c257a5dd1d2969cb5bfe893b1f1
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
from moviepy.editor import VideoFileClip
import cv2
import glob
import os
from deepface import DeepFace
import shutil
import numpy as np
from moviepy.editor import *
import math

23-12-23 13:40:00 - Directory /root/.deepface created
23-12-23 13:40:00 - Directory /root/.deepface/weights created


## Reducing FPS without Video Processing

In [3]:

def reduce_fps(video_path, output_directory, new_fps):

    # Extract file name
    file_name = os.path.basename(video_path)

    # Load the video
    video = VideoFileClip(video_path)

    # print the original FPS
    print(f"Orignal FPS: {video.fps}")

    # Reduce the FPS
    video = video.set_fps(new_fps)

    # Write the modified video
    output_path = os.path.join(output_directory, file_name)
    video.write_videofile(output_path, fps=new_fps)


In [4]:
input_video = r"/content/input_video.mp4"
output_directory = r"/content/Reduced FPS Videos"
new_fps = 10

reduce_fps(input_video,
           output_directory,
           new_fps)

Orignal FPS: 29.92
Moviepy - Building video /content/Reduced FPS Videos/input_video.mp4.
MoviePy - Writing audio in input_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/Reduced FPS Videos/input_video.mp4



Moviepy - Done !
Moviepy - video ready /content/Reduced FPS Videos/input_video.mp4


## Reducing FPS of Videos with Special Characters in Names

In [5]:
input_video = r"/content/---#input_video.mp4"
output_directory = r"/content/Reduced FPS Videos"
new_fps = 10

reduce_fps(input_video,
           output_directory,
           new_fps)

# the following error is intentional for demonstration purpose

Orignal FPS: 29.92
Moviepy - Building video /content/Reduced FPS Videos/---#input_video.mp4.
MoviePy - Writing audio in ---#input_videoTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/294 [00:00<?, ?it/s, now=None]

OSError: ignored

In [6]:

def reduce_fps(video_path, output_directory, new_fps):
    # Extract file name
    file_name = os.path.basename(video_path)

    # Load the video
    video = VideoFileClip(video_path)

    # Reduce the FPS
    video = video.set_fps(new_fps)

    # Prepare the output path with a temporary file name
    temp_output_path = os.path.join(output_directory, "temp.mp4")
    print(f"Temporary output path is: {temp_output_path}")

    # Write the modified video to the temporary file
    video.write_videofile(temp_output_path, fps=new_fps)

    # Prepare the final output path
    final_output_path = os.path.join(output_directory, file_name)
    print(f"Final output path is: {final_output_path}")

    # Rename the temporary file to the final file name
    shutil.move(temp_output_path, final_output_path)


In [7]:
input_video = r"/content/---#input_video.mp4"
output_directory = r"/content/Reduced FPS Videos"
new_fps = 10

reduce_fps(input_video,
           output_directory,
           new_fps)


chunk:   0%|          | 1/294 [00:23<1:53:40, 23.28s/it, now=None]

Temporary output path is: /content/Reduced FPS Videos/temp.mp4
Moviepy - Building video /content/Reduced FPS Videos/temp.mp4.
MoviePy - Writing audio in tempTEMP_MPY_wvf_snd.mp3



chunk:   0%|          | 1/294 [00:23<1:54:50, 23.52s/it, now=None]

MoviePy - Done.
Moviepy - Writing video /content/Reduced FPS Videos/temp.mp4




chunk:   0%|          | 1/294 [00:24<1:59:22, 24.44s/it, now=None]

Moviepy - Done !
Moviepy - video ready /content/Reduced FPS Videos/temp.mp4
Final output path is: /content/Reduced FPS Videos/---#input_video.mp4


## Reducing Number of Frames With Video Processing

In [8]:
def reduce_total_frames(video_path, directory, frames_to_capture):


    faces = []
    cap = cv2.VideoCapture(video_path)  # Read video file
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print("==================================")
    print(f"Original total number of frames: {total_frames}")
    print("===================================")

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = math.ceil(fps)

    path = os.path.basename(video_path)

    # Calculate interval for capturing frames

    interval = max(1, float(total_frames) / float(frames_to_capture))
    interval = np.round(interval)

    print("===============================")
    print(f"Interval to process frame : {interval}")
    print("===============================")

    frame_index = 0
    captured_frame_count = 0

    while cap.isOpened() and captured_frame_count < frames_to_capture:
        ret, frame = cap.read()

        if not ret:
            break

        # Capture frames at calculated intervals
        if frame_index % interval == 0:
            print(f"processing frame number {frame_index + 1}")
            try:
                face_props = DeepFace.extract_faces(img_path=frame,
                                                    target_size=(224, 224),
                                                    detector_backend="retinaface",
                                                    enforce_detection=False)

                if face_props:
                    frame = cv2.cvtColor(face_props[0]['face'], cv2.COLOR_BGR2RGB)
                    frame  = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    confidence = face_props[0]['confidence']
                    if confidence > 0.990:
                        features_dict = {"frames": frame, "confidence": confidence}
                        faces.append(features_dict)
                        captured_frame_count += 1
            except Exception as e:
                print(f"Error processing frame at index {frame_index}: {e}")


            if captured_frame_count % 10 == 0:
                print("============================")
                print(f"Total frames processed: {captured_frame_count}")
                print("============================")


        frame_index += 1

    image_frames = []

    for video_frame in faces:
        a = video_frame['frames']
        image_frame = np.interp(a, (a.min(), a.max()), (0, 255)).astype(np.uint8)
        image_frames.append(image_frame)

     # Create a video clip from the frames
    clip = ImageSequenceClip(image_frames, fps = fps)

    final_output_path = os.path.join(directory, path)

    clip.write_videofile(final_output_path)


In [10]:
input_video = r"/content/default_video.mp4"
output_directory = r"/content/Reduced FPS Videos"
num_frames = 100

reduce_total_frames(input_video,
                    output_directory,
                    num_frames)

Original total number of frames: 396
Interval to process frame : 4.0
processing frame number 1
processing frame number 5
processing frame number 9
processing frame number 13
processing frame number 17
processing frame number 21
processing frame number 25
processing frame number 29
processing frame number 33
processing frame number 37
Total frames processed: 10
processing frame number 41
processing frame number 45
processing frame number 49
processing frame number 53
processing frame number 57
processing frame number 61
processing frame number 65
processing frame number 69
processing frame number 73
processing frame number 77
Total frames processed: 20
processing frame number 81
processing frame number 85
processing frame number 89
processing frame number 93
processing frame number 97
processing frame number 101
processing frame number 105
processing frame number 109
processing frame number 113
processing frame number 117
Total frames processed: 30
processing frame number 121
processing


chunk:   0%|          | 1/294 [02:33<12:30:42, 153.73s/it, now=None]

t:   2%|▏         | 2/99 [00:50<40:27, 25.02s/it, now=None]

Moviepy - Building video /content/Reduced FPS Videos/default_video.mp4.
Moviepy - Writing video /content/Reduced FPS Videos/default_video.mp4





t:   0%|          | 0/99 [00:00<?, ?it/s, now=None]

t:  97%|█████████▋| 96/99 [00:00<00:00, 952.02it/s, now=None]


chunk:   0%|          | 1/294 [02:33<12:31:42, 153.93s/it, now=None]

t:   2%|▏         | 2/99 [00:50<40:37, 25.13s/it, now=None]

Moviepy - Done !
Moviepy - video ready /content/Reduced FPS Videos/default_video.mp4
